<a href="https://colab.research.google.com/github/kirankumarreddii/software-engineer-assessment/blob/main/software_engineer_assessment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
import re
import google.auth
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
from google.colab import auth
from google.auth.transport.requests import Request
from google.auth.credentials import Credentials
from google.oauth2.credentials import Credentials
from google.auth.exceptions import DefaultCredentialsError

def authenticate_google_docs():
    """Authenticate and return Google Docs API service."""
    auth.authenticate_user()
    creds, _ = google.auth.default()
    return build("docs", "v1", credentials=creds)

def create_google_doc(title="Product Team Sync"):
    """Create a new Google Doc and return its document ID."""
    try:
        service = authenticate_google_docs()
        doc = service.documents().create(body={"title": title}).execute()
        return doc['documentId']
    except HttpError as error:
        print(f"An error occurred: {error}")
        return None

def format_google_doc(doc_id, content):
    """Format and populate the Google Doc with structured content."""
    service = authenticate_google_docs()
    requests = []

    lines = content.split('\n')
    for line in lines:
        if line.startswith('# '):  # Heading 1 (Title)
            requests.append({"insertText": {"location": {"index": 1}, "text": line[2:] + "\n"}})
            requests.append({"updateParagraphStyle": {
                "range": {"startIndex": 1, "endIndex": len(line[2:]) + 1},
                "paragraphStyle": {"namedStyleType": "HEADING_1"},
                "fields": "namedStyleType"  # Specify the field to update
            }})
        elif line.startswith('## '):  # Heading 2
            requests.append({"insertText": {"location": {"index": 1}, "text": line[3:] + "\n"}})
            requests.append({"updateParagraphStyle": {
                "range": {"startIndex": 1, "endIndex": len(line[3:]) + 1},
                "paragraphStyle": {"namedStyleType": "HEADING_2"},
                "fields": "namedStyleType"  # Specify the field to update
            }})
        elif line.startswith('### '):  # Heading 3
            requests.append({"insertText": {"location": {"index": 1}, "text": line[4:] + "\n"}})
            requests.append({"updateParagraphStyle": {
                "range": {"startIndex": 1, "endIndex": len(line[4:]) + 1},
                "paragraphStyle": {"namedStyleType": "HEADING_3"},
                "fields": "namedStyleType"  # Specify the field to update
            }})
        elif line.startswith('- [ ] '):  # Checkbox
            text = line[6:]
            requests.append({"insertText": {"location": {"index": 1}, "text": text + "\n"}})
            requests.append({"updateParagraphStyle": {
                "range": {"startIndex": 1, "endIndex": len(text) + 1},
                "paragraphStyle": {"namedStyleType": "NORMAL_TEXT"},
                "fields": "namedStyleType"  # Specify the field to update
            }})
        elif '@' in line:  # Highlight mentions
            mentions = re.findall(r'(@\w+)', line)
            for mention in mentions:
                line = line.replace(mention, f"*{mention}*")
            requests.append({"insertText": {"location": {"index": 1}, "text": line + "\n"}})
        else:  # Normal text
            requests.append({"insertText": {"location": {"index": 1}, "text": line + "\n"}})

    service.documents().batchUpdate(documentId=doc_id, body={"requests": requests}).execute()
# if __name__ == "__main__":
#     markdown_notes = """# Product Team Sync - May 15, 2023
#     ## Attendees
#     - Sarah Chen (Product Lead)
#     - Mike Johnson (Engineering)
#     - Anna Smith (Design)
#     - David Park (QA)

#     ## Agenda

#     ### 1. Sprint Review
#     * Completed Features
#       * User authentication flow
#       * Dashboard redesign
#       * Performance optimization
#         * Reduced load time by 40%
#         * Implemented caching solution
#     """

#     document_id = create_google_doc()
#     if document_id:
#         format_google_doc(document_id, markdown_notes)
#         print(f"Google Doc created: https://docs.google.com/document/d/{document_id}/edit")


In [12]:
if __name__ == "__main__":
    markdown_notes = """# Product Team Sync - May 15, 2023
    ## Attendees
    - Sarah Chen (Product Lead)
    - Mike Johnson (Engineering)
    - Anna Smith (Design)
    - David Park (QA)

    ## Agenda

    ### 1. Sprint Review
    * Completed Features
      * User authentication flow
      * Dashboard redesign
      * Performance optimization
        * Reduced load time by 40%
        * Implemented caching solution
    """

    document_id = create_google_doc()
    if document_id:
        format_google_doc(document_id, markdown_notes)
        print(f"Google Doc created: https://docs.google.com/document/d/{document_id}/edit")


Google Doc created: https://docs.google.com/document/d/1-tC2EWkINYIrkwStKCtjseZOIVzPGT00V3k5E5CT4kY/edit
